# Hybrid Search using RRF

In this example we'll use the reciprocal rank fusion algorithm to combine the results of BM25 and kNN semantic search.
We'll use the same dataset we used in our [quickstart](https://github.com/joemcelroy/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/00-quick-start.ipynb) guide.



# 🧰 Requirements

For this example, you will need:

- Python 3.6 or later
- An Elastic deployment with minimum **4GB machine learning node**
   - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?elektra=en-ess-sign-up-page))
- The [ELSER](https://www.elastic.co/guide/en/machine-learning/8.8/ml-nlp-elser.html) model installed on your Elastic deployment
- The [Elastic Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/installation.html)


# Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Under **Advanced settings**, go to **Machine Learning instances**
   - You'll need at least **4GB** RAM per zone for this tutorial
   - Select **Create deployment**

# Install packages and initialize the Elasticsearch Python client

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`


In [55]:
!pip install -e git+https://github.com/elastic/elasticsearch-py.git@v8.8.2#egg=elasticsearch
!pip install sentence_transformers
!pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Obtaining elasticsearch from git+https://github.com/elastic/elasticsearch-py.git@v8.8.2#egg=elasticsearch
  Updating ./src/elasticsearch clone (to revision v8.8.2)
  Running command git fetch -q --tags
  Running command git reset --hard -q ec28599ac3ad1848f7c9d24cfad6f64cf8dc36ed
  Attempting uninstall: elasticsearch
    Found existing installation: elasticsearch 8.8.2
    Uninstalling elasticsearch-8.8.2:
      Successfully uninstalled elasticsearch-8.8.2
  Running setup.py develop for elasticsearch
You should consider upgrading via the '/Users/liamthompson/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.
huggingface/tokenizers: The current 

[TODO: Update]
Next we need to import the `elasticsearch` module and the `getpass` module.
`getpass` is part of the Python standard library and is used to securely prompt for credentials.

In [56]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import getpass
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Now we can instantiate the Python Elasticsearch client.
First we prompt the user for their password and Cloud ID.

🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [57]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

Confirm that the client has connected with this test

In [58]:
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': '9dd1e5c0b0d64796b8cf0746cf63d734', 'cluster_uuid': 'VeYvw6JhQcC3P-Q1-L9P_w', 'version': {'number': '8.9.0-SNAPSHOT', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ac7d79178c3e57c935358453331efe9e9cc5104d', 'build_date': '2023-06-21T09:08:25.219504984Z', 'build_snapshot': True, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0', 'transport_version': '8500019'}, 'tagline': 'You Know, for Search'}


Refer to https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect to a self-managed deployment.

Read https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new to learn how to connect using API keys.


# Create Elasticsearch index with required mappings

We need to add a field to support dense vector storage and search.
Note the `title_vector` field below, which is used to store the dense vector representation of the `title` field.



In [59]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"},
            "title_vector": { 
                "type": "dense_vector", 
                "dims": 384, 
                "index": "true", 
                "similarity": "dot_product" 
            }
        }
    }
}

# Create the index
client.indices.create(index='rrf_book_index', body=mapping)


/var/folders/vz/v2f6_x6s0kg51j2vbm5rlhww0000gn/T/ipykernel_2383/1628078329.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(index='rrf_book_index', body=mapping)


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [rrf_book_index/Ip8zitwhSMe0OJtEwpuqzQ] already exists')

## Dataset

Let's index some data.
Note that we are embedding the `title` field using the sentence transformer model.
Once indexed, you'll see that your documents contain a `title_vector` field (`"type": "dense_vector"`) which contains a vector of floating point values.
This is the embedding of the `title` field in vector space.
We'll use this vector to perform semantic search using kNN.

In [17]:
books = [
    {
        "title": "The Pragmatic Programmer: Your Journey to Mastery",
        "authors": ["andrew hunt", "david thomas"],
        "summary": "A guide to pragmatic programming for software engineers and developers",
        "publish_date": "2019-10-29",
        "num_reviews": 30,
        "publisher": "addison-wesley"
    },
    {
        "title": "Python Crash Course",
        "authors": ["eric matthes"],
        "summary": "A fast-paced, no-nonsense guide to programming in Python",
        "publish_date": "2019-05-03",
        "num_reviews": 42,
        "publisher": "no starch press"
    },
    {
        "title": "Artificial Intelligence: A Modern Approach",
        "authors": ["stuart russell", "peter norvig"],
        "summary": "Comprehensive introduction to the theory and practice of artificial intelligence",
        "publish_date": "2020-04-06",
        "num_reviews": 39,
        "publisher": "pearson"
    },
    {
        "title": "Clean Code: A Handbook of Agile Software Craftsmanship",
        "authors": ["robert c. martin"],
        "summary": "A guide to writing code that is easy to read, understand and maintain",
        "publish_date": "2008-08-11",
        "num_reviews": 55,
        "publisher": "prentice hall"
    },
    {
        "title": "You Don't Know JS: Up & Going",
        "authors": ["kyle simpson"],
        "summary": "Introduction to JavaScript and programming as a whole",
        "publish_date": "2015-03-27",
        "num_reviews": 36,
        "publisher": "oreilly"
    },
    {
        "title": "Eloquent JavaScript",
        "authors": ["marijn haverbeke"],
        "summary": "A modern introduction to programming",
        "publish_date": "2018-12-04",
        "num_reviews": 38,
        "publisher": "no starch press"
    },
    {
        "title": "Design Patterns: Elements of Reusable Object-Oriented Software",
        "authors": ["erich gamma", "richard helm", "ralph johnson", "john vlissides"],
        "summary": "Guide to design patterns that can be used in any object-oriented language",
        "publish_date": "1994-10-31",
        "num_reviews": 45,
        "publisher": "addison-wesley"
    },
    {
        "title": "The Clean Coder: A Code of Conduct for Professional Programmers",
        "authors": ["robert c. martin"],
        "summary": "A guide to professional conduct in the field of software engineering",
        "publish_date": "2011-05-13",
        "num_reviews": 20,
        "publisher": "prentice hall"
    },
    {
        "title": "JavaScript: The Good Parts",
        "authors": ["douglas crockford"],
        "summary": "A deep dive into the parts of JavaScript that are essential to writing maintainable code",
        "publish_date": "2008-05-15",
        "num_reviews": 51,
        "publisher": "oreilly"
    },
    {
        "title": "Introduction to the Theory of Computation",
        "authors": ["michael sipser"],
        "summary": "Introduction to the theory of computation and complexity theory",
        "publish_date": "2012-06-27",
        "num_reviews": 33,
        "publisher": "cengage learning"
    },
]

ObjectApiResponse({'took': 29, 'errors': False, 'items': [{'index': {'_index': 'rrf_book_index', '_id': '7c-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 10, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '7s-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 11, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '78-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 12, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '8M-QKokBaD3r4jKCZkdN', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 13, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': '8c-QKokBaD3r4jKCZkdN', '_version': 1, 'resu

## Index documents

Our dataset is a Python list that contains dictionaries of movie titles and descriptions.
We'll use the `helpers.bulk` method to index our documents in batches.

The following code iterates over the list of books and creates a list of actions to be performed.
Each action is a dictionary containing an "index" operation on our Elasticsearch index.
The book's title is encoded using our selected model, and the encoded vector is added to the book document.
The book document is then added to the list of actions.

Finally, we call the `bulk` method, specifying the index name and the list of actions.

In [60]:
actions = []
for book in books:
    actions.append({"index": {"_index": "rrf_book_index"}})
    titleEmbedding = model.encode(book["title"]).tolist()
    book["title_vector"] = titleEmbedding
    actions.append(book)

client.bulk(index="rrf_book_index", operations=actions)

ObjectApiResponse({'took': 26, 'errors': False, 'items': [{'index': {'_index': 'rrf_book_index', '_id': 'HM-NK4kBaD3r4jKCb0iX', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 20, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'Hc-NK4kBaD3r4jKCb0iX', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 21, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'Hs-NK4kBaD3r4jKCb0iX', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 22, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'H8-NK4kBaD3r4jKCb0iX', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 23, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'rrf_book_index', '_id': 'IM-NK4kBaD3r4jKCb0iX', '_version': 1, 'resu

## Pretty printing Elasticsearch responses

This is a helper function to print Elasticsearch responses in a readable format.

In [61]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nScore: {score}")
        print(pretty_output)

# Hybrid search using RRF


[Reciprocal Rank Fusion (RRF)](https://www.elastic.co/guide/en/elasticsearch/reference/current/rrf.html) is a state-of-the-art ranking algorithm for combining results from different information retrieval strategies.
RRF consistently improves the combined results of different search algorithms.
It outperforms all other ranking algorithms, and often surpasses the best individual results, without calibration.

⛔️ Because RRF is not yet available in the Python client, we'll use the `requests` library to make the request.⛔️ 

Check out the [RRF API reference](https://www.elastic.co/guide/en/elasticsearch/reference/master/rrf.html#rrf-api) for more information.






In [64]:
body = {
    "knn": {
        "field": "title_vector",
        "query_vector" : model.encode("python programming").tolist(),
        "k": 5,
        "num_candidates": 10},
        
}

response = client.search(index="rrf_book_index", body=body)

pretty_response(response)


ID: EuRpJYkBM7k8MFjPzXnU
Publication date: 2019-05-03
Title: Python Crash Course
Summary: A fast-paced, no-nonsense guide to programming in Python
Score: 0.8450546

ID: 7s-QKokBaD3r4jKCZkdN
Publication date: 2019-05-03
Title: Python Crash Course
Summary: A fast-paced, no-nonsense guide to programming in Python
Score: 0.8450546

ID: Hc-NK4kBaD3r4jKCb0iX
Publication date: 2019-05-03
Title: Python Crash Course
Summary: A fast-paced, no-nonsense guide to programming in Python
Score: 0.8450546

ID: EeRpJYkBM7k8MFjPzXnU
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Score: 0.6832866

ID: 7c-QKokBaD3r4jKCZkdN
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Score: 0.6832866


/var/folders/vz/v2f6_x6s0kg51j2vbm5rlhww0000gn/T/ipykernel_2383/1894029983.py:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="rrf_book_index", body=body)


In [65]:
body = {
    "query": {
        "term": {
            "summary": "shoes"
        }
    },
    "knn": {
        "field": "title_vector",
        "query_vector" : model.encode("python programming").tolist(),
        "k": 5,
        "num_candidates": 10},
    "rank": {
        "rrf": {
            "window_size": 50,
            "rank_constant": 20
        }
    }
}

response = client.search(index="rrf_book_index", body=body)

print(response)

/var/folders/vz/v2f6_x6s0kg51j2vbm5rlhww0000gn/T/ipykernel_2383/1777345508.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="rrf_book_index", body=body)


TypeError: search() got an unexpected keyword argument 'rank'